In [1]:
import geopandas as gpd
import gemgis as gg
from scipy.spatial import ConvexHull, convex_hull_plot_2d

In [2]:
thresholds=[300, 600]

In [3]:
gdf = gpd.read_file('../../Data/SWISSBUILDINGS3D_2_0_CHLV95LN02_1150-11.dxf')
gdf

,Layer,PaperSpace,SubClasses,Linetype,EntityHandle,Text,geometry
0,Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,70,None,MULTIPOLYGON Z (((2657460.991 1216407.070 616....
1,Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,DE,None,MULTIPOLYGON Z (((2658161.531 1216808.966 542....
2,Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,10A,None,MULTIPOLYGON Z (((2657477.949 1216827.938 582....
3,Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,160,None,MULTIPOLYGON Z (((2658708.284 1216295.554 546....
4,Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,1D3,None,MULTIPOLYGON Z (((2657656.504 1217295.045 545....
...,...,...,...,...,...,...,...
1874,Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,237B4,None,MULTIPOLYGON Z (((2657493.305 1217350.345 551....
1875,Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,237C6,None,MULTIPOLYGON Z (((2658578.884 1216375.768 542....
1876,Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,237F8,None,MULTIPOLYGON Z (((2657576.400 1217407.150 539....
1877,Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,238E4,None,MULTIPOLYGON Z (((2655776.906 1215425.252 785....


In [39]:
def getPoints(gdf):
    geom = gdf["geometry"]
    return [v for g in geom for v in g.exterior.coords]

def computeVolumeHull(gdf):
    hull = ConvexHull(gdf["points"])
    return hull.volume

def getGround(gdf):
    return min([v[2] for v in gdf["points"]])

def getBoundingBox(gdf):
    geom = gdf["geometry"]
    xmin = geom.bounds[0]
    ymin = geom.bounds[1]
    xmax = geom.bounds[2]
    ymax = geom.bounds[3]
    zs = [v[2] for v in gdf["points"]]
    return [[xmin, ymin, min(zs)], [xmax, ymax, max(zs)]]

def getGroundCenter(gdf):
    bb = gdf["bounding_box"]
    return [(bb[1][0]+bb[0][0])*1/2, (bb[1][1]+bb[0][1])*1/2, gdf["z_ground"]]

In [40]:
gdf["points"] = gdf.apply(getPoints, axis=1)
gdf["volume"] = gdf.apply(computeVolumeHull, axis=1)
gdf["z_ground"] = gdf.apply(getGround, axis=1)
gdf["bounding_box"] = gdf.apply(getBoundingBox, axis=1)
gdf["ground_center"] = gdf.apply(getGroundCenter, axis=1)

In [43]:
cleaned_df = gdf.loc[:,["volume", "bounding_box", "ground_center", "Layer", "EntityHandle"]]
cleaned_df.to_csv("volume_and_locations.csv")
cleaned_df

,volume,bounding_box,ground_center,Layer,PaperSpace,SubClasses,Linetype,EntityHandle,Text
0,2338.01,"[[2657457.662999999, 1216397.927999999, 616.06...","[2657467.5764999986, 1216407.845999999, 616.06...",Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,70,None
1,403.42,"[[2658155.833000001, 1216799.57, 542.455000000...","[2658159.8820000016, 1216804.7880000006, 542.4...",Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,DE,None
2,1112.02,"[[2657472.517999999, 1216821.289000001, 582.62...","[2657480.534, 1216829.3585, 582.6230000000069]",Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,10A,None
3,1489.51,"[[2658703.905000001, 1216292.114, 546.93899999...","[2658711.1400000006, 1216299.5539999995, 546.9...",Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,160,None
4,1330.31,"[[2657652.613000002, 1217280.982999999, 545.34...","[2657658.561000001, 1217288.171, 545.347999999...",Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,1D3,None
...,...,...,...,...,...,...,...,...,...
1874,51.64,"[[2657492.813000001, 1217347.510000002, 551.92...","[2657494.2705000006, 1217348.9275000007, 551.9...",Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,237B4,None
1875,1342.36,"[[2658564.399999999, 1216366.669, 542.32899999...","[2658572.692, 1216374.9415000011, 542.32899999...",Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,237C6,None
1876,1592.91,"[[2657573.408, 1217398.465, 539.4590000000026]...","[2657586.6889999993, 1217406.1555000006, 539.4...",Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,237F8,None
1877,1576.30,"[[2655775.559999999, 1215411.103, 785.62099999...","[2655781.8204999994, 1215419.1779999994, 785.6...",Gebaeude Einzelhaus,None,AcDbEntity:AcDbPolyFaceMesh,None,238E4,None
